# 상장기업정보 수집하기

In [1]:
import pandas as pd
import time
import requests
import re
from bs4 import BeautifulSoup as bs

In [ ]:
dict(method=searchCorpList, pageIndex=1, currentPageSize=15, comAbbrv=, beginIndex=, orderMode=3, orderStat=D, isurCd=, repIsuSrtCd=, searchCodeType=, marketType=, searchType=13, industry=, fiscalYearEnd=all, comAbbrvTmp=, location=all)

In [6]:
url ="https://kind.krx.co.kr/corpgeneral/corpList.do"
payload = dict(method="searchCorpList", pageIndex=1, currentPageSize=100, orderMode=3, orderStat="D", searchType=13, fiscalYearEnd="all", location="all")
r = requests.post(url, data=payload)
print(r.url)
print(r.status_code)
soup = bs(r.content, 'lxml')
soup

https://kind.krx.co.kr/corpgeneral/corpList.do
200


<html><body><section class="scrarea type-00">
<table class="list type-00 tmt30" summary="회사명, 업종, 주요제품, 상장일, 결산월, 대표자명, 홈페이지, 지역">
<caption>목록</caption>
<colgroup>
<col width="*"/>
<col width="15%"/>
<col width="20%"/>
<col width="11%"/>
<col width="8%"/>
<col width="10%"/>
<col width="8%"/>
<col width="12%"/>
</colgroup>
<thead>
<tr class="first" id="title-contents">
</tr>
</thead>
<tbody>
<tr class="first">
<td class="first" title="명인제약(주)"><img alt="유가증권" class="vmiddle legend" src="/images/common/icn_t_yu.gif"/> <a href="#companysum" id="companysum" onclick="companysummary_open('31745'); return false;" title="명인제약"> 명인제약</a> </td>
<td class="textOverflow" title="의약품 제조업">의약품 제조업</td>
<td class="textOverflow" title="신경정신계열, 구강계열 완제의약품">신경정신계열, 구강계열 완제의약품</td>
<td class="txc">2025-10-01</td>
<td class="txc">12월</td>
<td class="txc" title="이행명">이행명</td>
<td class="txc">
<a class="btn ico homepage" href="http://www.myunginph.co.kr" target="_blank" title="http://www.myunginph.co.kr"><sp

In [15]:
total_page = int(soup.select_one(".info.type-00 > em").text.replace(",", "")) // 100 + 1

In [19]:
# 증권종류
soup.select_one(".vmiddle.legend")['alt']

'유가증권'

In [23]:
# 회사명
soup.select_one("#companysum")['title']

'명인제약'

In [39]:
# 종목코드
code = soup.select_one("#companysum")["onclick"].split("'")[1]
code = f'{code:0<6}'
code

'317450'

In [77]:
def text_clean(text):
    result = re.sub("[^가-힣a-zA-Z0-9]", " ", text).replace("  ", " ").replace("  ", " ").strip()
    return result

In [117]:
def info_extraction(trs):
    result = []
    for row in trs:
        for idx, td in enumerate(row.select('td')):
            if idx == 0:
                kind = td.select_one("img")['alt']
                com_name = td['title']
                code = td.select_one("a")["onclick"].split("'")[1]
                code = f"{code:0<6}"
            elif idx == 1:
                biz_type = text_clean(td.text)
            elif idx == 2:
                product = text_clean(td.text)
            elif idx == 3:
                resi_date = td.text
            elif idx == 4:
                settlement = td.text
            elif idx == 5:
                ceo = td.text
            elif idx == 6:
                homepage = td.select_one("a")['href'] if td.select_one("a") != None else "홈페이지 없음"
            elif idx == 7:
                region = td.text
        temp = (kind, com_name, code, biz_type, product, resi_date, settlement, ceo, homepage, region)
        result.append(temp)
    return result


In [118]:
import pandas as pd
import time
import requests
import re
from bs4 import BeautifulSoup as bs

In [119]:
result = {}
col = ['주식종목', '회사이름', '종목코드', '업종', '주요제품', '상장일', '결산월', '대표자명',
      '홈페이지', '지역']
page = 1
total_page = 1

while page <= total_page:
    print(f"{page}/{total_page} 자료 수집중            ", end="\r")
    url ="https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method="searchCorpList", pageIndex=page, currentPageSize=100, orderMode=3, orderStat="D", searchType=13, fiscalYearEnd="all", location="all")
    r = requests.post(url, data=payload)
#     print(r.url)
#     print(r.status_code)
    soup = bs(r.content, 'lxml')
    if total_page == 1:
        total_page = int(soup.select_one(".info.type-00 > em").text.replace(",", "")) // 100 + 1
    values = info_extraction(soup.select("tbody > tr"))
    for value in values:
        for key, item in zip(col, value):
            result.setdefault(key, []).append(item)

    page += 1
         
df = pd.DataFrame(result)
df

https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgeneral/corpList.do
200
https://kind.krx.co.kr/corpgene

,주식종목,회사이름,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,유가증권,명인제약(주),317450,의약품 제조업,신경정신계열 구강계열 완제의약품,2025-10-01,12월,이행명,http://www.myunginph.co.kr,경기도
1,코스닥,케이비제33호기업인수목적(주),0072Z0,금융 지원 서비스업,기업인수합병,2025-09-30,12월,박성원,홈페이지 없음,서울특별시
2,코스닥,삼성기업인수목적11호(주),0071M0,금융 지원 서비스업,기업인수합병,2025-09-30,12월,문소연,홈페이지 없음,서울특별시
3,코스닥,주식회사 에스투더블유,488280,소프트웨어 개발 및 공급업,공공 정부기관용 사이버안보 빅데이터 플랫폼 기업 및 기관용 사이버위협 인텔리전스 플...,2025-09-19,12월,서상덕,http://s2w.inc/,경기도
4,코스닥,오아 주식회사,342870,가전제품 및 정보통신장비 소매업,계절 생활 주방 건강가전 및 헬스 건강식품,2025-09-09,12월,김상무,http://www.oa-world.com,서울특별시
...,...,...,...,...,...,...,...,...,...,...
2758,유가증권,(주)유한양행,000100,의약품 제조업,의약품 삐콤씨 안티푸라민 렉라자 로수바미브 코푸시럽 등 생활용품 유한락스 해피홈 유...,1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
2759,유가증권,씨제이대한통운(주),000120,도로 화물 운송업,Contract Logistics 포워딩 항만하역 해운 택배국제특송 SCM Cons...,1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
2760,유가증권,(주)경방,000050,종합 소매업,섬유류 면사 면혼방사 면직물 면혼방직물 화섬사 화섬직물 제조 도매 수출입,1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
2761,유가증권,(주)유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시
